# Add correlations to independent fission yield covariance matrix

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
%%capture --no-stdout
import sandy

## Retrive nuclear data file

First, we define the energy and zam that we want to analyze and for the chosen zam we get the evaluated nuclear data file - in this case U-235 from ENDF/B-VIII.0 - using get_endf6_file:

In [ ]:
e = 2.53000e-02
zam = 922350
tape = sandy.get_endf6_file('endfb_80','nfpy', zam)

Then we can read the decay data information stored in the ENDF6 file and extract the values of the fission yields and their associated uncertainties:

In [ ]:
nfpy = sandy.Fy.from_endf6(tape)
nfpy.data.set_index(["MAT","MT", "ZAM", "ZAP"])

## Obtain covariance matrix  

Assuming the uncertaities equal to the standard deviations, we can build the diagonal covariance matrix as follows:

In [ ]:
conditions = {'ZAM': zam, "E": e}
fy_data = nfpy._filters(conditions).data.set_index('ZAP')[['MT', 'DFY']]
Vx_prior = fy_data.query('MT==454').DFY
Vx_prior = sandy.CategoryCov.from_var(Vx_prior)
Vx_prior.data.head()

We want to update the variance matrices of the independent fission yields using the method of generalised least squares (GLS). It is an adjustment technique that states that the information on some prior system parameters can be improved with the addition of newknowledge for which relationships between data and parameters are established. These relationships, or constraints, must be linearised in the form:

$$
y - y_a = S \cdot (\theta -\theta_a) 
$$

where $\theta$ are the parameters of the system, $\theta_a$ the prior estimates of $\theta$, $y$ the responses of the constraining  equation,  $y_a$ the responses of the constraining equation to the prior  estimates $\theta_a$ and S are the sensitivity coefficients of the response $y – y_a$ to the parameters $\theta – \theta_a$. 

It is assumed that no correlations existed between the prior and the new information. Then, further information $\eta$ could be introduced in order to derive refined values for the parameters $\theta$,  with all the available uncertainty information properly incorporated into the formalism. The updating process is the following:

$$
\theta - \theta_a = V_a - S^T \cdot (S \cdot V_a \cdot S^T + V)^{-1} \cdot (\eta - y_a)
$$

$$
V_s = V_a - V_a \cdot S^T \cdot \left(S\cdot V_a \cdot S^T + V \right)^{-1} \cdot S \cdot V_a
$$

where $V_a$ is the covariance matrix of the prior estimates of the  parameters $\theta$, $V$ is the covariance matrix of the introduced data fitting the constraining system $\eta$, and $V_s$ is the updated  covariance matrix of the system parameters ($\theta$).

A more complete overview of the the  followed GLS technique is described in https://doi.org/10.1016/j.anucene.2015.10.027

To perform this updating process we exploit the relationship between the independent fission yield and the $\textit{chain ﬁssion yield ch(A)}$, which in matrix form is:

$$
D^T \cdot IFY = Ch 
$$

The chain fission yield may be confused with the $\textit{mass ﬁssion yield M(A)}$, indeed the two can differ by a few percent. The design matrix $S$ in this case is $D^T$, while the parameters $\theta$ are the independent fission yields $IFY$ and the response is the vector with the chain ﬁssion yields $Ch$ 

By using evaluated chain ﬁssion yields to modify independent ﬁssion yield data, we assume that we have a deeper knowledge on the ﬁrst. This is a consistent assumption since 
the chain ﬁssion yield and uncertainties are evaluated mostly directly from the measurements while the independent ﬁssion yields are not.

The extra information for the evaluation of $V$ are obtained from the IAEA document  [Evaluation and Compilation of Fission Product Yields 1993](https://www-nds.iaea.org/endf349/la-ur-94-3106.pdf) (page 18-29)

In SANDY two options were implemented to retrive the design matrix $D^T$ and both of them are showed in this notebook.

- ### First option: `kind='mass yield'`

With this option we assume that the chain fission yield is equal to the mass yield, so we don't take into account the delayed neutron emission. All the necessary information to calculate the $D^T$ matrix are stored in the `sandy.Fy` object. For this reason, if the `kind='mass yield'` option is selected, the only extra parameters that we have to pass to the method `sandy.Fy.cov_update()` are the zam and energy chosen for the analysis.

In [ ]:
cov_massyield = nfpy.cov_update(zam, e, kind='mass yield')
cov_massyield.data.head()

- ### Second option: `kind='chain yield'` 

With this option we evaluate the $D^T$ matrix which respects the constrain $\sum_{A_i=A}y(A_i) = ch(A)$. To do so, we also need the information stored in `sandy.DecayData` instance, so it must be passed as further parameter to the `sandy.Fy.cov_update()` method.

In [ ]:
tape_rdd = sandy.get_endf6_file('endfb_80', 'decay', 'all')

In [ ]:
rdd = sandy.DecayData.from_endf6(tape_rdd)

In [ ]:
cov_chainyield = nfpy.cov_update(zam, e, kind='chain yield', decay_data=rdd)
cov_chainyield.data.head()

## Visual comparison of correlation matrices 

A visual comparison of the correlation matrices obtained with the GLS method is showed above. Some nuclides were selected among the zap numbers to have a more clear visualization of the added correlations.

In [ ]:
zap =pd.Index([
    551480,
    551490,
    561480,
    561490,
    571480,
    571490,
    581480,
    591480,
    591481,
    601480,
])
nuclide = pd.Series(zap.values, index = zap, name="ZAP").to_frame()
nuclide['nuclide']= nuclide.ZAP.apply(sandy.zam.zam2latex)
nuclide_index = nuclide.nuclide.values
cov_my_plot = sandy.CategoryCov(cov_massyield.data.loc[zap, zap].set_index(nuclide_index).T.set_index(nuclide_index))
cov_cy_plot = sandy.CategoryCov(cov_chainyield.data.loc[zap, zap].set_index(nuclide_index).T.set_index(nuclide_index))
Vx_prior_plot = sandy.CategoryCov(Vx_prior.data.loc[zap, zap].set_index(nuclide_index).T.set_index(nuclide_index))

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(17, 5), dpi=100)
ax1, ax2 , ax3 = axes

sns.heatmap(Vx_prior_plot.get_corr(),  cmap="bwr", vmin=-1, vmax=1, ax=ax1)
ax1.set_title(f' From prior covariance matrix of IFY')

sns.heatmap(cov_my_plot.get_corr(), cmap="bwr", vmin=-1, vmax=1, ax=ax2)
ax2.set_title(f' From GLS update covariance matrix of IFY with `kind=mass yield`')

sns.heatmap(cov_cy_plot.get_corr(),  cmap="bwr", vmin=-1, vmax=1, ax=ax3)
ax3.set_title(f' From GLS update covariance matrix of IFY with `kind=chain yield`')

fig.tight_layout()

Obtained the covariance matrix, we can go ahead with the sampling procedure. In this notebook we use the updated covariace matrix obtained with the first option but the steps to follow are the same whatever covariance matrix is selected.

## Relative covariance matrix

We prefer to work with relative covariance matrix to be consistent with the covariance matrices stored in the ENDF-6 format. SANDY can work with relative or absolute covariance matrix thanks to the `relative` kwarg option inserted in the `sandy.CategoryCov.sampling()` method. Now we apply the $sandwitch \ rule$ with the diagonal sensitivity matrix equals to the collection of the best estimates of the fission yields to have the updated relative covariance matrix.

In [ ]:
conditions = {'ZAM': zam, "E": e, "MT": 454}
S = np.diag(nfpy._filters(conditions).data.FY)

In [ ]:
idx = cov_massyield.data.index
cov_relative = sandy.CategoryCov(pd.DataFrame(cov_massyield.sandwich(S).data.values, index=idx, columns=idx))
cov_relative.data.head()

## Obtain perturbation coefficient

In [ ]:
nsmp = 500 
coeff = cov_relative.sampling(nsmp, pdf='normal', relative=True, tolerance=0)
coeff.data.head()

## Apply first perturbation coefficient to fission yields

This step will be iteratively repeted for each perturbation coefficient to obtain nsmp perturbed fission yields. The perturbation coefficients are given as ratio values, e.g., 1.05 for a perturbation of +5%.

In [ ]:
nfpy_new = nfpy.custom_perturbation(coeff.data.iloc[0,:], zap=list(coeff.data.iloc[0,:].index), e=e, mt=454)

## Create an ENDF6 file with the perturbed nuclear data

In [ ]:
tape_new = nfpy_new.to_endf6(tape)
file = tape_new.to_file("perturbed_fy")